In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/CS-520

/content/drive/MyDrive/CS-520


In [ ]:
!git clone https://github.com/Avantikaa/CS-520.git

Cloning into 'CS-520'...
remote: Enumerating objects: 96, done.
remote: Counting objects: 100% (96/96), done.
remote: Compressing objects: 100% (85/85), done.
remote: Total 96 (delta 20), reused 54 (delta 7), pack-reused 0
Unpacking objects: 100% (96/96), done.


In [4]:
%cd mlops/
!ls

/content/drive/MyDrive/CS-520/mlops
520_Project.ipynb      CS-520		  wandb_test_dataset.pkl
cifar10		       __pycache__	  wandb_test_results.pkl
cifar_train_mlflow.py  wandb
cifar_train_wandb.py   wandb_checkpoints


In [5]:
!pip install pytorch-lightning
!pip install torch
!pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 798 kB 35.4 MB/s 
     |████████████████████████████████| 125 kB 78.5 MB/s 
     |████████████████████████████████| 512 kB 79.0 MB/s 
     |████████████████████████████████| 87 kB 7.5 MB/s 
  Created wheel for fire: filename=fire-0.4.0-py2.py3-none-any.whl size=115943 sha256=827b6d1abe1f75819e3782fe0b8bf1083442e984c0728d28a8626680b8e15927
  Stored in directory: /root/.cache/pip/wheels/1f/10/06/2a990ee4d73a8479fe2922445e8a876d38cfbfed052284c6a1
Successfully built fire
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.9 MB 35.2 MB/s 
     |████████████████████████████████| 168 kB 71.4 MB/s 
     |████████████████████████████████| 182 kB 74.0 MB/s 
     |███████████

In [24]:
sweep_config = {
    'method': 'random'
    }
metric = {
    'name': 'val_loss_avg',
    'goal': 'minimize'   
    }

sweep_config['metric'] = metric

parameters_dict = {
    'epochs': {
        'values': [6,12,18]
        },
     'lr':{
         'values':[0.1,0.5,0.9]
        },
      'batch_size': {
          'values':[64,128,256]
      },  

    }
sweep_config['parameters'] = parameters_dict

In [14]:
import wandb

In [30]:
sweep_id = wandb.sweep(sweep_config, project="CS-520-mlops")

Create sweep with ID: twy4dzbp
Sweep URL: https://wandb.ai/520_saga/CS-520-mlops/sweeps/twy4dzbp


In [26]:
import pytorch_lightning as pl
# your favorite machine learning tracking tool
from pytorch_lightning.loggers import WandbLogger

import torch
# print("GPUs available: ",torch.cuda.device_count())
from torch import nn
from torch.nn import functional as F
from torch.utils.data import random_split, DataLoader

from torchmetrics import Accuracy

from torchvision import transforms
from torchvision.datasets import CIFAR10
import copy
import argparse

from cifar10 import utils
from cifar10.module import Module

import warnings
import wandb
import os
os.environ['WANDB_API_KEY'] = '2c3e9b6c4f20687980e9cb2da43e2432c149f55a' #add your account's API key
wandb.login()

warnings.filterwarnings("ignore")

utils.set_seed(0)

In [27]:
class ImagePredictionLogger(pl.callbacks.Callback):
    def __init__(self, val_samples, num_samples=32):
        super().__init__()
        self.num_samples = num_samples
        self.val_imgs, self.val_labels = val_samples
        self.metrics = []
    
    def on_validation_epoch_end(self, trainer, pl_module):
        # Bring the tensors to CPU
        val_imgs = self.val_imgs.to(device=pl_module.device)
        val_labels = self.val_labels.to(device=pl_module.device)
        # Get model prediction
        each_me = copy.deepcopy(trainer.callback_metrics)
        self.metrics.append(each_me)
        logits = pl_module(val_imgs)
        preds = torch.argmax(logits, -1)
        # Log the images as wandb Image
        trainer.logger.experiment.log({
            "Sample Validation Set Performance":[wandb.Image(x, caption=f"Predicted:{pred}, True:{y}") 
                           for x, pred, y in zip(val_imgs[:self.num_samples], 
                                                 preds[:self.num_samples], 
                                                 val_labels[:self.num_samples])]
            })

In [28]:
def train():
    parser = argparse.ArgumentParser(add_help=False)

    parser.add_argument("--exp-name", default="Default")
    parser.add_argument("--num-epochs", default=6, type=int)
    parser.add_argument("--lr", default=0.1)
    parser.add_argument("--batch_size", default=32)
    parser.add_argument("--tags", action=utils.DictPairParser, metavar=utils.DictPairParser.METAVAR)

    parser = Module.add_model_specific_args(parser)
    #hparams = parser.parse_args()
    hparams, unknown = parser.parse_known_args()
    print("hello")
    #Namespace(batch_size=128, data_root='.dataset/cifar10', exp_name='Default', model_name='vgg16', num_epochs=6, tags=None
    
    
    
    with wandb.init():
        wandb.config.epochs = hparams.num_epochs
        wandb.config.lr = hparams.lr
        wandb.config.batch_size = hparams.batch_size
        config = wandb.config
        cifar_module = Module(hparams)
        #--------------Wandb functionality------------------
        # dm = CIFAR10DataModule(batch_size=32)
        # # To access the x_dataloader we need to call prepare_data and setup.
        # dm.prepare_data()
        # dm.setup()
        
        # Samples required by the custom ImagePredictionLogger callback to log image predictions.
        val_samples = next(iter(cifar_module.val_dataloader()))
        val_imgs, val_labels = val_samples[0], val_samples[1]
        #----------------------end--------------------------
        
        wandb_logger = WandbLogger(project='wandb-lightning', job_type='train')
    
        #exp = wandb_logger.experiment.get_experiment_by_name(hparams.exp_name)
        #artifacts_dir = os.path.join(exp.artifact_location, wandb_logger.run_id, "artifacts")
    
        checkpoint_callback = pl.callbacks.ModelCheckpoint(
            dirpath='./wandb_checkpoints/', save_top_k=-1, verbose=True, monitor="val_loss_avg", mode="min"
        )
        early_stop_callback = pl.callbacks.EarlyStopping(monitor="val_loss")
        
        ipl = ImagePredictionLogger(val_samples)

        trainer = pl.Trainer(
            logger=wandb_logger, callbacks=[early_stop_callback,ipl,checkpoint_callback], max_epochs=config.epochs, accelerator="cuda", auto_lr_find=True, auto_scale_batch_size=None
        )

        loss_logs = ipl.metrics
        for l in loss_logs:
          print(l)
          wandb.log({'val_loss_avg':l['val_loss_avg']})

        trainer.fit(cifar_module)
        predictions = trainer.predict(cifar_module, cifar_module.test_dataloader())
        torch.save(predictions, "wandb_test_results.pkl")
        torch.save(cifar_module.test_dataloader().dataset, "wandb_test_dataset.pkl")
        
        data_artifact = wandb.Artifact('wandb_test_dataset.pkl',type='dataset')
        data_artifact.add_file(local_path='wandb_test_dataset.pkl')
        wandb.log_artifact(data_artifact)
        
        results_artifact = wandb.Artifact('wandb_test_results.pkl',type='dataset')
        results_artifact.add_file(local_path='wandb_test_results.pkl')
        wandb.log_artifact(results_artifact)

In [ ]:
wandb.agent(sweep_id, train, count=10)

wandb: Agent Starting Run: 7pc4xnhj with config:
wandb: 	batch_size: 256
wandb: 	epochs: 18
wandb: 	lr: 0.9


hello


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


wandb: WARNING Config item 'epochs' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'lr' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'batch_size' was locked by 'sweep' (ignored update).


Files already downloaded and verified


...

Files already downloaded and verified


INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name       | Type             | Params
------------------------------------------------
0 | _model     | VGG              | 134 M 
1 | _criterion | CrossEntropyLoss | 0     
------------------------------------------------
134 M     Trainable params
0         Non-trainable params
134 M     Total params
537.206   Total estimated model params size (MB)
wandb: WARNING Config item 'lr' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'batch_size' was locked by 'sweep' (ignored update).


Sanity Checking: 0it [00:00, ?it/s]

Files already downloaded and verified
Files already downloaded and verified


Training: 0it [00:00, ?it/s]